In [ ]:
# --- SETUP AWAL ---
# Python ≥3.5 diperlukan
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 diperlukan
import sklearn
assert sklearn.__version__ >= "0.20"

# Impor library umum
import numpy as np
import pandas as pd
import os

# Untuk membuat plot yang konsisten
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Direktori untuk menyimpan gambar
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Menyimpan gambar", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

"""
# Bab 6: Decision Trees

## Penjelasan Teoretis: Konsep Dasar Decision Tree
Decision Trees (Pohon Keputusan) adalah algoritma Machine Learning yang serbaguna, mampu melakukan tugas klasifikasi, regresi, dan bahkan multioutput. Mereka adalah model yang kuat, mampu menyesuaikan diri dengan dataset yang kompleks.

Decision Trees juga merupakan komponen fundamental dari Random Forests, salah satu algoritma ML paling kuat yang ada saat ini.

Salah satu keunggulan Decision Trees adalah mereka memerlukan sangat sedikit persiapan data. Mereka tidak memerlukan penskalaan fitur atau pemusatan data sama sekali. Model ini sering disebut sebagai model *white box* karena cara kerjanya sangat intuitif dan mudah diinterpretasikan.
"""

# --- 1. Melatih dan Memvisualisasikan Decision Tree ---
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

# Melatih model Decision Tree dengan kedalaman maksimum 2
tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

"""
#### Visualisasi Pohon
Kita bisa memvisualisasikan pohon yang telah dilatih menggunakan `export_graphviz` dari Scikit-Learn untuk membuat file definisi grafik (.dot).
"""
from sklearn.tree import export_graphviz

export_graphviz(
        tree_clf,
        out_file=os.path.join(IMAGES_PATH, "iris_tree.dot"),
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

# Untuk mengonversi file .dot ke PNG, Anda bisa menggunakan tool Graphviz dari command line:
# $ dot -Tpng iris_tree.dot -o iris_tree.png
# Atau, kita bisa menampilkannya langsung di notebook jika graphviz terinstal.
try:
    import graphviz
    with open(os.path.join(IMAGES_PATH, "iris_tree.dot")) as f:
        dot_graph = f.read()
    display(graphviz.Source(dot_graph))
except ImportError:
    print("Harap instal graphviz untuk menampilkan pohon keputusan di notebook.")


"""
### Membuat Prediksi
Untuk membuat prediksi, kita mulai dari *root node* (node paling atas) dan menelusuri pohon ke bawah. Setiap node mengajukan pertanyaan tentang sebuah fitur. Berdasarkan jawabannya, kita pindah ke *child node* kiri atau kanan, hingga mencapai *leaf node* (node daun, yang tidak memiliki anak). Kelas yang diprediksi adalah kelas mayoritas di leaf node tersebut.

#### Atribut Node:
- **`samples`**: Jumlah instance pelatihan yang berlaku pada node ini.
- **`value`**: Jumlah instance pelatihan dari setiap kelas yang berlaku pada node ini.
- **`gini`**: Ukuran *impurity* (ketidakmurnian) sebuah node. Node murni (`gini=0`) jika semua instance di dalamnya berasal dari kelas yang sama.
"""

# --- 2. Mengestimasi Probabilitas Kelas ---
# Decision Tree juga bisa mengestimasi probabilitas sebuah instance.
# Ini adalah rasio instance dari setiap kelas di leaf node tempat instance tersebut jatuh.
print("\nProbabilitas untuk bunga [5cm, 1.5cm]:", tree_clf.predict_proba([[5, 1.5]]))
print("Prediksi kelas:", tree_clf.predict([[5, 1.5]]))


# --- 3. Algoritma Pelatihan CART ---
"""
Scikit-Learn menggunakan algoritma **Classification And Regression Tree (CART)** untuk melatih Decision Trees.
- Algoritma ini bekerja dengan membagi set pelatihan menjadi dua subset menggunakan satu fitur `k` dan sebuah *threshold* `tk`.
- Ia mencari pasangan `(k, tk)` yang menghasilkan subset paling murni (diukur dengan Gini atau Entropy), dengan mempertimbangkan ukuran subset.
- Proses ini dilakukan secara rekursif hingga mencapai kedalaman maksimum atau tidak dapat menemukan pemisahan yang mengurangi ketidakmurnian.

CART adalah **greedy algorithm** (algoritma serakah). Ia membuat pemisahan optimal di setiap level, tetapi tidak menjamin akan menemukan pohon yang optimal secara global.
"""

# --- 4. Hyperparameter Regularisasi ---
"""
Decision Tree sangat fleksibel dan dapat dengan mudah **overfitting** jika tidak dibatasi. Untuk menghindarinya, kita perlu melakukan regularisasi dengan membatasi kebebasan pohon.

Hyperparameter regularisasi utama:
- `max_depth`: Kedalaman maksimum pohon.
- `min_samples_split`: Jumlah minimum sampel yang harus dimiliki sebuah node sebelum bisa dipecah.
- `min_samples_leaf`: Jumlah minimum sampel yang harus dimiliki oleh sebuah leaf node.
- `max_leaf_nodes`: Jumlah maksimum leaf node.
- `max_features`: Jumlah maksimum fitur yang dievaluasi untuk pemisahan di setiap node.

Menaikkan `min_*` atau mengurangi `max_*` akan meregularisasi model.
"""
# Contoh efek regularisasi pada moons dataset
from sklearn.datasets import make_moons
Xm, ym = make_moons(n_samples=100, noise=0.25, random_state=53)

deep_tree_clf1 = DecisionTreeClassifier(random_state=42) # Tanpa batasan
deep_tree_clf2 = DecisionTreeClassifier(min_samples_leaf=4, random_state=42) # Dengan regularisasi
deep_tree_clf1.fit(Xm, ym)
deep_tree_clf2.fit(Xm, ym)

# Fungsi untuk plot decision boundary
def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.45, -1, 1.5], alpha=0.5, contour=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = mpl.colors.ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = mpl.colors.ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", alpha=alpha)
    plt.axis(axes)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

plt.figure(figsize=(11, 4))
plt.subplot(121)
plot_decision_boundary(deep_tree_clf1, Xm, ym, axes=[-1.5, 2.45, -1, 1.5], alpha=0.2)
plt.title("Tanpa Batasan (Overfitting)", fontsize=16)
plt.subplot(122)
plot_decision_boundary(deep_tree_clf2, Xm, ym, axes=[-1.5, 2.45, -1, 1.5], alpha=0.2)
plt.title("Regularisasi min_samples_leaf=4", fontsize=16)
plt.show()


# --- 5. Regresi dengan Decision Tree ---
"""
Decision Tree juga bisa melakukan regresi. Alih-alih memprediksi kelas, setiap leaf node memprediksi sebuah nilai. Nilai prediksi ini adalah rata-rata dari nilai target semua instance pelatihan di leaf node tersebut.

Algoritma CART bekerja dengan cara yang sama, tetapi sekarang ia mencoba membagi set pelatihan untuk meminimalkan **Mean Squared Error (MSE)**, bukan Gini impurity.
"""
from sklearn.tree import DecisionTreeRegressor

# Buat dataset kuadratik dengan noise
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X, y)

# --- 6. Instabilitas Decision Tree ---
"""
Decision Tree memiliki beberapa keterbatasan:
1. **Sensitif terhadap rotasi data**: Pohon keputusan menyukai batas keputusan ortogonal (tegak lurus sumbu), sehingga kinerjanya bisa buruk jika data dirotasi.
2. **Sangat sensitif terhadap variasi kecil dalam data pelatihan**: Menghapus satu instance saja bisa menghasilkan pohon yang sangat berbeda. Ini membuat model menjadi tidak stabil.

Random Forests (dibahas di Bab 7) mengatasi masalah ketidakstabilan ini dengan merata-ratakan prediksi dari banyak pohon.
"""

# --- JAWABAN LATIHAN TEORETIS ---

"""
### Latihan: Jawaban Teoretis

**1. Berapa perkiraan kedalaman Decision Tree yang dilatih (tanpa batasan) pada set pelatihan dengan satu juta instance?**
Kedalaman sebuah Decision Tree yang seimbang adalah sekitar $log_2(m)$, di mana $m$ adalah jumlah instance.
$log_2(1,000,000) = log(10^6)/log(2) \approx 6 \times 2.3 / 0.69 \approx 20$.
Jadi, kedalamannya sekitar 20.

**2. Apakah Gini impurity sebuah node umumnya lebih rendah atau lebih besar dari induknya? Apakah selalu lebih rendah/besar?**
Gini impurity sebuah node **umumnya lebih rendah** dari induknya. Algoritma CART secara spesifik mencari pemisahan yang meminimalkan Gini impurity (tertimbang). Oleh karena itu, jumlah Gini impurity dari *child nodes* (setelah dihitung rata-rata tertimbang berdasarkan jumlah sampel) **selalu lebih rendah** dari Gini impurity *parent node*-nya. Namun, Gini impurity dari satu *child node* tunggal bisa saja lebih tinggi dari induknya.

**3. Jika Decision Tree mengalami overfitting, apakah ide yang bagus untuk mencoba mengurangi `max_depth`?**
**Ya, sangat bagus**. Mengurangi `max_depth` adalah salah satu cara paling umum dan efektif untuk meregularisasi Decision Tree dan mengurangi overfitting. Ini membatasi kompleksitas model.

**4. Jika Decision Tree mengalami underfitting, apakah ide yang bagus untuk mencoba penskalaan fitur input?**
**Tidak**. Salah satu keunggulan Decision Tree adalah ia tidak terpengaruh oleh penskalaan fitur. Penskalaan tidak akan membantu mengatasi underfitting. Untuk mengatasi underfitting, Anda harus mencoba meningkatkan kompleksitas model, misalnya dengan menaikkan `max_depth` atau melonggarkan hyperparameter regularisasi lainnya (mengurangi `min_*`).

**5. Jika butuh satu jam untuk melatih Decision Tree pada 1 juta instance, berapa perkiraan waktu untuk melatihnya pada 10 juta instance?**
Kompleksitas pelatihan CART adalah sekitar $O(n \times m \log(m))$. Jika kita asumsikan `n` (jumlah fitur) tetap, rasio waktunya adalah:
$T(10m) / T(m) = (10m \log(10m)) / (m \log(m)) = 10 \times \log(10m) / \log(m)$
$\log(10m) = \log(10) + \log(m)$
Rasio = $10 \times (\log(10) + \log(m)) / \log(m) = 10 \times (1 + \log(10)/\log(m))$
Untuk $m=10^6$, $\log(m)=6$.
Rasio $\approx 10 \times (1 + 1/6) \approx 11.67$.
Jadi, perkiraan waktunya adalah $1 \text{ jam} \times 11.67 \approx 11.7$ jam.

**6. Jika set pelatihan Anda berisi 100.000 instance, apakah `presort=True` akan mempercepat pelatihan?**
**Tidak**. `presort=True` hanya mempercepat pelatihan pada dataset kecil (kurang dari beberapa ribu instance). Untuk 100.000 instance, pengaturan ini justru akan memperlambat pelatihan secara signifikan.

"""

# --- LATIHAN PRAKTIS ---

"""
### Latihan 7: Latih dan fine-tune Decision Tree untuk moons dataset.
"""
print("\n--- Latihan 7: Fine-tuning Decision Tree ---")
from sklearn.model_selection import train_test_split, GridSearchCV

# a. Buat dataset
X_moons, y_moons = make_moons(n_samples=10000, noise=0.4, random_state=42)

# b. Bagi dataset
X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons, test_size=0.2, random_state=42)

# c. Gunakan GridSearchCV untuk mencari hyperparameter terbaik
params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)
print("Hyperparameter terbaik:", grid_search_cv.best_params_)

# d. Latih model dengan hyperparameter terbaik dan evaluasi
from sklearn.metrics import accuracy_score

best_tree = grid_search_cv.best_estimator_
y_pred = best_tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi pada test set: {accuracy * 100:.2f}%")

"""
### Latihan 8: Tumbuhkan sebuah "forest" (Hutan).
"""
print("\n--- Latihan 8: Membuat Random Forest Manual ---")
from sklearn.model_selection import ShuffleSplit

# a. Buat 1000 subset dari training set
n_trees = 1000
n_instances = 100

mini_sets = []
rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, _ in rs.split(X_train):
    mini_sets.append((X_train[mini_train_index], y_train[mini_train_index]))

# b. Latih 1000 Decision Tree
from sklearn.base import clone
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    y_pred_tree = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred_tree))

print(f"Akurasi rata-rata dari 1000 pohon: {np.mean(accuracy_scores) * 100:.2f}%")


# c. Buat prediksi mayoritas (majority-vote)
# Buat matriks prediksi dari semua pohon
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

# Gunakan mode (nilai yang paling sering muncul) untuk setiap instance
from scipy.stats import mode
y_pred_majority_votes, _ = mode(Y_pred, axis=0, keepdims=False)

# d. Evaluasi prediksi mayoritas
accuracy_forest = accuracy_score(y_test, y_pred_majority_votes)
print(f"Akurasi Random Forest manual: {accuracy_forest * 100:.2f}%")
print("Akurasi forest lebih tinggi dari pohon tunggal!")

